In [1]:
import pymongo
import datetime
import os
import subprocess
import signal
import time
from core.config import MONGODB_URL, DATABASE_NAME, CONFIG_COLLECTION, LABEL_RETRAIN_QUEUE_COLLECTION, API_PORT, API_HOST, API_WORKER, SLEEP_INTERVAL_SECOND, ANACONDA_ENV_NAME, PATH

client = pymongo.MongoClient(MONGODB_URL)

from utils.logs import config_log, change_service_status

config_col = client[DATABASE_NAME][CONFIG_COLLECTION]
trainer = config_col.find_one({"name": "trainer"})
trainer_obj = {
        "name": "trainer",
        "status": "down",
        "logs": [],
    }
if not trainer:
    config_col.insert_one(trainer_obj)

def run_trainer():
    return_status = "Finish."
    command = f"""eval "$(conda shell.bash hook)";
        conda activate {ANACONDA_ENV_NAME};
        python {PATH}/trainer.py;"""
    config_log("trainer", "Start Trainer")
    change_service_status("trainer", "up")
    process = subprocess.Popen(command, shell=True)
    while True:
        try:
            try:
                poll = process.poll()
                int(poll)
                if poll == 0:
                    return_status = f"Finish with code {poll}"
                if poll == 4:
                    return_status = f"Error with code {poll}, CUDA out of memory!"
                    time.sleep(600) #if out of memory, don't keep try it...
                else:
                    return_status = f"Error with code {poll}"
                #print(f"Process Stop Running, return code {poll}")
                break
            except TypeError:
                #print("Process Still Running")
                time.sleep(3)
                print("AA")
                continue
            
        except KeyboardInterrupt:
            print("KeyboardInterrupt, stop program")
            return_status = "KeyboardInterrupt"
            break

    config_log("trainer", f"Stop trainer because {return_status}")
    change_service_status("trainer", "down")
    process.send_signal(signal.SIGINT)
    process.wait()
    print(f"Stop trainer because {return_status}")
    return return_status

training_queue_col = client[DATABASE_NAME][LABEL_RETRAIN_QUEUE_COLLECTION]
while True:

    import re
    result = training_queue_col.find({
        "status":  re.compile("(training|waiting)")
    })

    result = list(result)

    if result:
        return_status = run_trainer()
        if return_status == "KeyboardInterrupt":
            break
        else:
            continue
    time.sleep(SLEEP_INTERVAL_SECOND)

KeyboardInterrupt, stop program
Stop trainer because KeyboardInterrupt


Traceback (most recent call last):
  File "/home/eason/Python/GSoC_Accord/API/trainer.py", line 4, in <module>
    import torch
  File "/home/eason/anaconda3/envs/adapter/lib/python3.7/site-packages/torch/__init__.py", line 196, in <module>
    from torch._C import *
RuntimeError: KeyboardInterrupt: 


In [2]:
import os
cwd = os.getcwd()

SyntaxError: invalid syntax (1936283382.py, line 2)

In [ ]:
cwd

In [ ]:
import pymongo
client = pymongo.MongoClient(MONGODB_URL)
training_queue_col = client[DATABASE_NAME][LABEL_RETRAIN_QUEUE_COLLECTION]

training_queue = training_queue_col.find({
    "status":  re.compile("(training|waiting)")
})
training_queue = list(training_queue)

while True:
    if True or len(training_queue) == 0:
        break

# Old Train First
training_queue.sort(key = lambda x: x["add_time"], reverse= False)

now_is_training = training_queue[0]

# Update parameter On Each Iter
train_data_search_filter = now_is_training["train_data_filter"]
label_name = now_is_training["label_name"]
Epoch_Times = now_is_training["epochs"]

In [ ]:
from utils.logs import trainer_log, queue_task_log

In [ ]:
training_queue_col = client[DATABASE_NAME][LABEL_RETRAIN_QUEUE_COLLECTION]
import re
result = training_queue_col.find({
    "status": {
        "$not": re.compile("done")}
})
result = list(result)

In [ ]:
type(result[0]["_id"])

In [ ]:
import os
import subprocess
import signal
import time

In [ ]:
command = "python trainer.py"
process = subprocess.Popen(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

In [ ]:
process.poll()

In [ ]:
# stop process
process.send_signal(signal.SIGINT)

In [ ]:
process.kill()

In [ ]:
type(process)

In [ ]:
process.pid

In [ ]:
while True:
    time.sleep(1)

In [ ]:
process